**Ejecute este script con el archivo que desde para añadir las nuevas columnas relacionadas al sentimiento del titulo de la noticia**

In [4]:
!pip3 install pysentimiento
import pandas as pd
from pysentimiento import create_analyzer

# 1. Cargar el CSV de test
df = pd.read_csv("./train.csv", sep=";")

def replace_nan_with_description(df):
    for index, row in df.iterrows():
        if pd.isna(row['Titulo']):
            description = row['Descripcion']
            if isinstance(description, str):
                first_sentence = description.split('.')[0]
                df.at[index, 'Titulo'] = first_sentence
    return df

df = replace_nan_with_description(df)

# 1. Crear el analyzer de sentimiento para español
analyzer = create_analyzer(task="sentiment", lang="es")

# 2. Función para extraer el label principal y las probabilidades
def get_sentiment_info(text):
    if not isinstance(text, str):
        text = ""  # manejo de posibles NaN o datos vacíos
    prediction = analyzer.predict(text)
    return prediction.output, prediction.probas

# 4. Aplicar la función a la columna 'Texto'
df["sentiment_result"] = df["Titulo"].apply(lambda x: get_sentiment_info(x)[0])
df["sentiment_prob"] = df["Titulo"].apply(lambda x: get_sentiment_info(x)[1])

# 5. Extraer probabilidades individuales
df["sentiment_prob_neg"] = df["sentiment_prob"].apply(lambda d: d["NEG"])
df["sentiment_prob_neu"] = df["sentiment_prob"].apply(lambda d: d["NEU"])
df["sentiment_prob_pos"] = df["sentiment_prob"].apply(lambda d: d["POS"])

df.drop(columns=['sentiment_prob', 'sentiment_result'], inplace=True)

df.to_csv("train_modificado.csv", index=False, sep=";")  # Usa punto y coma como separador
df.head()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,ID,Label,Titulo,Descripcion,Fecha,sentiment_prob_neg,sentiment_prob_neu,sentiment_prob_pos
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,0.339239,0.423685,0.237076
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,0.606836,0.296518,0.096645
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,0.282778,0.523354,0.193868
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,0.351684,0.537340,0.110976
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,0.598948,0.336025,0.065027
